## Preamble

In [1]:
import numpy as np
import pandas as pd
import polars as pl
import pyarrow as pa
import datetime as dt
from arcticdb import Arctic, LibraryOptions, QueryBuilder, LazyDataFrame, OutputFormat
from arcticdb.version_store.library import Library
from arcticdb.version_store.processing import ExpressionNode
from arcticdb_ext.version_store import OperationType
from typing import Iterator

In [2]:
def print_table(table):
    print(pl.from_arrow(table))

## ArcticDB data

In [3]:
test_uri = "lmdb://tmp/polars_integration"
test_lib_name = "polars"
test_symbol = "pls"
ac = Arctic(test_uri)

In [4]:
ac.delete_library(test_lib_name)
lib = ac.create_library(test_lib_name)

In [5]:
df = pd.DataFrame({"col1": np.arange(10, dtype=np.int64), "col2": np.arange(10, 20, dtype=np.float64)})
lib.write(test_symbol, df)
df

,col1,col2
0,0,10.0
1,1,11.0
2,2,12.0
3,3,13.0
4,4,14.0
5,5,15.0
6,6,16.0
7,7,17.0
8,8,18.0
9,9,19.0


In [6]:
lazy_frame = lib.read(test_symbol, output_format=OutputFormat.EXPERIMENTAL_ARROW, lazy = True)

In [7]:
print_table(lazy_frame.collect().data)

shape: (10, 2)
┌──────┬──────┐
│ col1 ┆ col2 │
│ ---  ┆ ---  │
│ i64  ┆ f64  │
╞══════╪══════╡
│ 0    ┆ 10.0 │
│ 1    ┆ 11.0 │
│ 2    ┆ 12.0 │
│ 3    ┆ 13.0 │
│ 4    ┆ 14.0 │
│ 5    ┆ 15.0 │
│ 6    ┆ 16.0 │
│ 7    ┆ 17.0 │
│ 8    ┆ 18.0 │
│ 9    ┆ 19.0 │
└──────┴──────┘


In [8]:
q = QueryBuilder()
q = q[(q["col1"] > 2) & (q["col2"] < 18)]
lazy_frame2 = lib.read(test_symbol, query_builder = q, output_format=OutputFormat.EXPERIMENTAL_ARROW, lazy = True)
print_table(lazy_frame2.collect().data)

shape: (5, 2)
┌──────┬──────┐
│ col1 ┆ col2 │
│ ---  ┆ ---  │
│ i64  ┆ f64  │
╞══════╪══════╡
│ 3    ┆ 13.0 │
│ 4    ┆ 14.0 │
│ 5    ┆ 15.0 │
│ 6    ┆ 16.0 │
│ 7    ┆ 17.0 │
└──────┴──────┘


## Polars expression AST parser

In [9]:
import ast
import re
from typing import Any, Optional


class PolarsToArcticDBTranslator:
    """
    Translates Polars expressions to ArcticDB QueryBuilder operations.
    
    Usage:
        translator = PolarsToArcticDBTranslator()
        qb = translator.translate(polars_expr, query_builder)
    """

    def translate(self, polars_expr: str, query_builder: Any) -> Any:
        """
        Translate a Polars expression string to ArcticDB QueryBuilder operations.
        
        Args:
            polars_expr: String representation of Polars expression
            query_builder: ArcticDB QueryBuilder instance
            
        Returns:
            Modified QueryBuilder instance
        """
        
        # Clean the expression - remove surrounding brackets if present
        expr = polars_expr.strip()
        if expr.startswith('[') and expr.endswith(']'):
            expr = expr[1:-1].strip()

        expr = self._replace_square_brackets(expr)
        
        # Preprocess to handle Polars-specific notation like [dyn int: 2]
        expr = self._preprocess_expression(expr)

         # Parse the expression
        try:
            tree = ast.parse(expr, mode='eval')
            expr_node = self._process_node(tree.body)
        except SyntaxError as e:
            raise ValueError(f"Invalid Polars expression: {polars_expr}") from e
        
        return query_builder[expr_node]
        
    def _replace_square_brackets(self, text: str) -> str:
        while True:
            close = text.rfind('])')
            if close == -1:
                break
            open_ = text.rfind('([', 0, close)
            if open_ == -1:
                break
            # Replace the matched ([...]) with (...)
            text = text[:open_] + '(' + text[open_ + 2:close] + ')' + text[close + 2:]
        return text
    
    def _preprocess_expression(self, expr: str) -> str:
        """
        Preprocess Polars expression to handle special notation.
        
        Converts patterns like [dyn int: 2] to just the value (2).
        """
        # Pattern to match [dyn type: value] or [lit type: value]
        pattern = r'[\[\(](dyn|lit)\s+\w+:\s*([^\]\)]+)[\]\)]'
        
        def replace_dynamic(match):
            return match.group(2).strip()
        
        return re.sub(pattern, replace_dynamic, expr)

    def _process_node(self, node: ast.AST) -> Any:
        """Process an AST node and apply corresponding ArcticDB operation."""

        node_type = type(node)
        match node_type:
            case ast.Call:
                return self._process_call(node)
            case ast.Attribute:
                return self._process_attribute(node)
            case ast.Name:
                return node.id
            case ast.Constant:
                return node.value
            case ast.Compare:
                return self._process_compare(node)
            case ast.BinOp:
                return self._process_binop(node)
            case ast.UnaryOp:
                return self._process_unaryop(node)
            #case ast.List:
            #    return [self._process_node(elt) for elt in node.elts]
            #case ast.Tuple:
            #    return tuple(self._process_node(elt) for elt in node.elts)
            case _:
                raise NotImplementedError(f"Node type {node_type.__name__} not supported")

    def _process_call(self, node: ast.Call) -> Any:
        """Process function calls (e.g., pl.col(), methods)."""

        func_type = type(node.func)
        match func_type:
            case ast.Attribute:
                # TODO: rework this
                # Method call like pl.col('x').sum()
                #obj = self._process_node(node.func.value)
                #method = node.func.attr
                #args = [self._process_node(arg) for arg in node.args]
                #kwargs = {kw.arg: self._process_node(kw.value) for kw in node.keywords}
                
                #return self._apply_method(obj, method, args, kwargs)
                raise NotImplementedError(f"Node.func type ast.Attribute not supported")
            
            case ast.Name:
                # Function call like col('x')
                func_name = node.func.id
                args = [self._process_node(arg) for arg in node.args]
                
                if func_name == 'col':
                    return ExpressionNode.column_ref(args[0]) if args else None
            case _:
                return None

    def _process_attribute(self, node: ast.Attribute) -> Any:
        """Process attribute access like pl.col or obj.attr."""
        #TODO rework this
        obj = self._process_node(node.value)
        attr = node.attr

        # Handle pl.col pattern
        #if obj == 'pl' and attr == 'col':
        #    return 'col'
        
        #return f"{obj}.{attr}"
        raise NotImplementedError(f"{obj}.{attr}: Node type ast.Attribute not supported")

    def _process_compare(self, node: ast.Compare) -> Any:
        """Process comparison operations and apply filters."""
        
        left = self._process_node(node.left)
        
        # Handle multiple comparisons
        for op, comparator in zip(node.ops, node.comparators):
            right = self._process_node(comparator)
            expr_node = None
            op_type = type(op)

            match op_type:
                case ast.Eq:
                    expr_node = ExpressionNode.compose(left, OperationType.EQ, right)
                case ast.NotEq:
                    expr_node = ExpressionNode.compose(left, OperationType.NE, right)
                case ast.Lt:
                    expr_node = ExpressionNode.compose(left, OperationType.LT, right)
                case ast.LtE:
                    expr_node = ExpressionNode.compose(left, OperationType.LE, right)
                case ast.Gt:
                    expr_node = ExpressionNode.compose(left, OperationType.GT, right)
                case ast.GtE:
                    expr_node = ExpressionNode.compose(left, OperationType.GE, right)
                case ast.In:
                    expr_node = ExpressionNode.compose(left, OperationType.ISIN, right)
                case ast.NotIn:
                    expr_node = ExpressionNode.compose(left, OperationType.ISNOTIN, right)
                case _:
                    raise NotImplementedError(f"Operator {op_type} not supported")
            left = expr_node
        
        return expr_node

    def _process_binop(self, node: ast.BinOp) -> Any:
        """Process binary operations."""
        
        left = self._process_node(node.left)
        
        right = self._process_node(node.right)
        op_type = type(node.op)

        match op_type:
            case ast.Add:
                return ExpressionNode.compose(left, OperationType.ADD, right)
            case ast.Sub:
                return ExpressionNode.compose(left, OperationType.SUB, right)
            case ast.Mult:
                return ExpressionNode.compose(left, OperationType.MUL, right)
            case ast.Div:
                return ExpressionNode.compose(left, OperationType.DIV, right)
            # TODO: operator & and | can be used for boolean operations between
            # filters, or bitwise operations between integers values. The current
            # implementation supports boolean operations only (which should be
            # the most common case)
            case ast.BitOr:
                return ExpressionNode.compose(left, OperationType.OR, right)
            case ast.BitAnd:
                return ExpressionNode.compose(left, OperationType.AND, right)
            case _:
                raise NotImplementedError(f"Operator {op_type} not supported")
    
    def _process_unaryop(self, node: ast.UnaryOp) -> Any:
        """Process unary operations."""
        
        operand = self._process_node(node.operand)
        op_type = type(node.op)

        match op_type:
            case ast.Invert:
                return ExpressionNode.compose(operand, OperationType.NOT, None)
            case ast.USub:
                return ExpressionNode.compose(operand, OperationType.NEG, None)
            case _:
                raise NotImplementedError(f"Operator {op_type} not supported")
            
        return None

In [10]:
translator = PolarsToArcticDBTranslator()

In [11]:
qb = QueryBuilder()
qb = translator.translate(str(pl.col("col1") > 2), qb)

In [12]:
lazy_frame2 = lib.read(test_symbol, query_builder = qb, output_format=OutputFormat.EXPERIMENTAL_ARROW, lazy = True)
print_table(lazy_frame2.collect().data)

shape: (7, 2)
┌──────┬──────┐
│ col1 ┆ col2 │
│ ---  ┆ ---  │
│ i64  ┆ f64  │
╞══════╪══════╡
│ 3    ┆ 13.0 │
│ 4    ┆ 14.0 │
│ 5    ┆ 15.0 │
│ 6    ┆ 16.0 │
│ 7    ┆ 17.0 │
│ 8    ┆ 18.0 │
│ 9    ┆ 19.0 │
└──────┴──────┘


In [13]:
qb = QueryBuilder()
qb = translator.translate(str((pl.col("col1") > 2) & (pl.col("col2") < 18)), qb)

qe = QueryBuilder()
qe = qe[(qe["col1"] > 2) & (qe["col2"] < 18)]
qb == qe

True

In [14]:
qb = QueryBuilder()
qb = translator.translate(str(pl.col("col2") + pl.col("col2") + pl.col("col2")), qb)

qe = QueryBuilder()
qe = qe[qe["col2"] + qe["col2"] + qe["col2"]]
qb == qe

True

In [15]:
bmi_expr = pl.col("weight") / (pl.col("height") ** 2)
qb = QueryBuilder()
qb = translator.translate(str(bmi_expr), qb)

NotImplementedError: Column["height"].pow2: Node type ast.Attribute not supported

## Polars Plugin

In [16]:
def parse_schema(
    lib: Library,
    symbol: str,
    as_of: int | str | dt.datetime | None = None
) -> pl.Schema:
    arrow_df = lib.read(symbol, as_of=as_of, output_format=OutputFormat.EXPERIMENTAL_ARROW, row_range=((0,1))).data
    return pl.Schema(arrow_df.schema) 

In [17]:
def scan_arcticdb(
    uri: str,
    lib_name: str,
    symbol: str,
    as_of: int | str | dt.datetime | None = None
) -> pl.LazyFrame:

    ac = Arctic(uri)
    lib = ac.get_library(lib_name)

    schema = parse_schema(lib, symbol, as_of)

    def source_generator(
        with_columns: list[str] | None,
        predicate: pl.Expr | None,
        n_rows: int | None,
        batch_size: int | None
    ) -> Iterator[pl.DataFrame]:
        if predicate is not None:
            print(str(predicate))
        else:
            print("No Predicate")

        if with_columns:
            print(with_columns)
        else:
            print("No column")

        qb = None
        if predicate is not None:
            tl = PolarsToArcticDBTranslator()
            qb = QueryBuilder()
            qb = tl.translate(str(predicate), qb)
            
        # TODO: convert predicate to QueryBuilder and pass it to read
        lazy_df = lib.read(symbol, as_of = as_of, columns = with_columns, query_builder = qb, lazy = True, output_format=OutputFormat.EXPERIMENTAL_ARROW)

        if batch_size is None:
            batch_size = 1000

        if n_rows is not None:
            batch_size = min(batch_size, n_rows)
        
        read_idx = 0
        while n_rows is None or n_rows > 0:
            lazy_df_slice = lazy_df.row_range((read_idx, read_idx + batch_size))
            read_idx += batch_size
            arrow_df = lazy_df_slice.collect().data
            if n_rows is not None:
                n_rows -= arrow_df.num_rows
            elif arrow_df.num_rows < batch_size:
                n_rows = 0

            yield pl.from_arrow(arrow_df)

    return pl.io.plugins.register_io_source(io_source=source_generator, schema = schema)    

## Plugin tests

In [18]:
pl_frame = scan_arcticdb(test_uri, test_lib_name, test_symbol)

20251017 09:41:36.257244 7417 W arcticdb | LMDB path at /home/yoyo/dev/polars/notebooks/tmp/polars_integration/ has already been opened in this process which is not supported by LMDB. You should only open a single Arctic instance over a given LMDB path. To continue safely, you should delete this Arctic instance and any others over the LMDB path in this process and then try again. Current process ID=[7417]


In [19]:
pl_frame.collect()

No Predicate
No column


col1,col2
i64,f64
0,10.0
1,11.0
2,12.0
3,13.0
4,14.0
5,15.0
6,16.0
7,17.0
8,18.0


In [20]:
pl_frame.select(pl.col("col1") / pl.col("col2")).collect()

No Predicate
['col1', 'col2']


col1
f64
0.0
0.090909
0.166667
0.230769
0.285714
0.333333
0.375
0.411765
0.444444


In [21]:
pl_frame.filter(pl.col("col1") > 2).collect()

[(col("col1")) > (2)]
No column


col1,col2
i64,f64
3,13.0
4,14.0
5,15.0
6,16.0
7,17.0
8,18.0
9,19.0


In [22]:
pl_frame.filter((pl.col("col1") > 2) & (pl.col("col2") < 18)).collect()

[([(col("col2")) < (18.0)]) & ([(col("col1")) > (2)])]
No column


col1,col2
i64,f64
3,13.0
4,14.0
5,15.0
6,16.0
7,17.0
